In [1]:
import os

%pwd

'/Users/wilsvenleong/Downloads/learning-materials/mlflow/research'

In [2]:
os.chdir("../")  # Move up to root directory
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/mlflow'

## 1. Update `config.yaml`

```yaml
data_transformation:
  root_dir: artifacts/data_transformation
  unzip_data_path: artifacts/data_ingestion/winequality-red.csv
```

## 4. Update the Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    unzip_data_path: Path

## 5. Update the `ConfigurationManager` in `src/config/configuration.py`

In [4]:
from mlflow_project.constants import *
from mlflow_project.utils.common import read_yaml, create_directories

from mlflow_project import logger


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        """
        Creates the root directory and returns
        the configuration for data transformation.

        Returns:
            DataTransformationConfig: Configuration for data transformation.
        """
        data_transformation = self.config.data_transformation

        create_directories([data_transformation.root_dir])

        file_path = os.path.join(data_transformation.root_dir, ".gitkeep")
        if not os.path.exists(file_path):
            with open(file_path, "w") as f:
                logger.info(
                    f"Creating file: .gitkeep in directory {data_transformation.root_dir}"
                )
                pass

        data_transformation_config = DataTransformationConfig(
            root_dir=Path(data_transformation.root_dir),
            unzip_data_path=Path(data_transformation.unzip_data_path),
        )

        return data_transformation_config

## 6. Update the Components

In [5]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    # Different data transformation techniques such can be applied
    # Differrent kinds of EDA in the ML cycle can be performed here before passing the data to the model
    # Only `train_test_split` will be used here because this is a project focusing on MLOps
    def train_test_split(self) -> None:
        """Splits the data into train and test sets."""
        df = pd.read_csv(self.config.unzip_data_path)

        # Split the data into train and test sets -> 3 to 1 split
        train, test = train_test_split(df, train_size=0.75)
        logger.info("Successfully split data into train and test sets")

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info(f"Successfully saved the train set with shape: {train.shape}")
        logger.info(f"Successfully saved the test set with shape: {test.shape}")

## 7. Update the Pipeline

In [6]:
try:
    config_manager = ConfigurationManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_split()
except Exception as e:
    raise e

[ 2023-10-24 22:24:00,970 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: config/config.yaml
[ 2023-10-24 22:24:00,974 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: params.yaml
[ 2023-10-24 22:24:00,976 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: schema.yaml
[ 2023-10-24 22:24:00,977 ] 53 common mlflow_project -  INFO - Created directory at: artifacts
[ 2023-10-24 22:24:00,977 ] 53 common mlflow_project -  INFO - Created directory at: artifacts/data_transformation
[ 2023-10-24 22:24:01,002 ] 21 1941872955 mlflow_project -  INFO - Successfully split data into train and test sets
[ 2023-10-24 22:24:01,030 ] 26 1941872955 mlflow_project -  INFO - Successfully saved the train set with shape: (1199, 12)
[ 2023-10-24 22:24:01,030 ] 27 1941872955 mlflow_project -  INFO - Successfully saved the test set with shape: (400, 12)
